In [ ]:
!pip install pdfplumber camelot-py[cv] tabula-py pytesseract pdf2image pandas

     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 165.2 kB/s eta 0:00:01
     ------------------------------------ - 41.0/42.8 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 232.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
     ---------------------------------------- 48.2/48.2 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/60.2 kB ? eta -:--:--
   ---------------------------------------- 60.2/60.2 kB 3.1 MB/s eta 0:00:00
   ----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.2.4 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.2.4 which is incompatible.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
